In [1]:
import pandas as pd
import time
import csv

from tqdm.auto import tqdm

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key="gQK7DGi9o3fLJrl97qyrVGfBh28QP3FO",
    base_url="https://api.upstage.ai/v1/solar"
)

In [3]:
df_test = pd.read_csv("../data/test.csv")

In [4]:
df_train = pd.read_csv("../data/train.csv")

In [5]:
exam_prompt = "너는 이제 대화문을 간단하게 요약하는 직무를 맡은 사원이야\n대화문 예시와 요약문 예시를 줄테니 예시와 유사한 방식으로 대화문을 요약해 줘\n대화문 예시:\n"\
    + df_train["dialogue"][0] + "\n요약문 예시:\n" + df_train["summary"][0]

In [6]:
# this system prompt will be used globally on the chat session.
system_prompt = {
    "role": "system",
    "content": exam_prompt
}

In [7]:
df_test

,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, ..."
1,test_1,#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또...
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Per..."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person..."
4,test_4,#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금...
...,...,...
494,test_495,"#Person1#: 헤이, 찰리, 학교 끝나고 우리 집에 와서 나랑 비디오 게임 할..."
495,test_496,#Person1#: 어떻게 컨트리 음악에 관심을 가지게 되었나요?\r\n#Perso...
496,test_497,"#Person1#: 실례합니다, 앨리스. 이곳을 사용해본 적이 없는데, 기계를 어떻..."
497,test_498,#Person1#: 매튜? 안녕!\r\n#Person2#: 스티브! 오랜만이네! 얼...


In [8]:
df_test_dialogue = df_test["dialogue"]

In [9]:
data = list(zip(df_test["fname"], df_test["dialogue"]))

In [12]:
model = "solar-1-mini-chat"

dialogue_num = 1

# text_summary_prompt = f"내가 예시로 주는 대화문을 보고 대화문의 요약 예시와 유사하게 요약이 할 수 있도록 해당 대화문을 재구성해서 대화문을 {dialogue_num}개 만들어줘"
# text_summary_prompt = f'내가 위에 준 대화문 예시와 요약 예시을 보고 같은 방식으로 아래에 내가 준 대화문을 요약해 줘'
# exam_prompt = "대화문 예시 :\n" + df_train["dialogue"][0] + "\n\n" + "요약 예시 :\n" + df_train["summary"][0]
with open("inference_by_solar.txt", "a", newline="") as f:
    for fname ,dial in tqdm(data):
        chat_history = []
        history_size = 500
        
        for i in range(1, dialogue_num+1):
            # Step 1: Get user input.
            mk_fname = f"{fname}_{i}"
            user_prompt = dial
            chat_history.append({
                "role": "user",
                "content": user_prompt
            })
            
            # Step 2: Use the chat history to generate the chatbot's response.
            messages = [system_prompt] + chat_history
            stream = client.chat.completions.create(
                model=model,
                messages=messages,
                stream=True,
            )
            
            # Step 3: Output the Solar response chunks.
            solar_response = ""
            for chunk in stream:
                response_content = chunk.choices[0].delta.content
                if response_content is not None:
                    solar_response += response_content
            # Finish response output.
            f.write(fname+"<split>"+solar_response + "\n")

            # Step 4: Append the Solar response to the chat history.
            chat_history.append({
                "role": "assistant",
                "content": solar_response
            })
            time.sleep(2)
            
            

        # Step 5: Ensure the chat history doesn't exceed the size limit.
        chat_history = chat_history[:history_size]

  0%|          | 0/499 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'message': 'Too Many Requests'}